[View in Colaboratory](https://colab.research.google.com/github/mchirico/heart/blob/hr-9/HeartWithGsutil.ipynb)

In [0]:
import requests
import pandas as pd
import numpy as np 
from datetime import datetime



import warnings
warnings.filterwarnings("ignore")

try:
  import seaborn as sns
  import matplotlib.pyplot as plt
  import matplotlib.dates as mdates
  sns.set(style="white", color_codes=True)
  from matplotlib_venn import venn2
  import keras
  import pydot
  import libarchive
  import torch
except:
  print("Will try to install software")
  # Lines below .... Only need to be run once at the top of the notebook.
  !pip install -q matplotlib-venn
  !pip install -q keras
  # #   https://pypi.python.org/pypi/pydot
  !apt-get -qq install -y graphviz && pip install -q pydot
  !apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
  !pip install -U -q PyDrive
  !pip install --upgrade -q gspread
  
  !pip3 install http://download.pytorch.org/whl/cpu/torch-0.4.0-cp36-cp36m-linux_x86_64.whl 
  !pip3 install torchvision
  
  import seaborn as sns
  import matplotlib.pyplot as plt
  import matplotlib.dates as mdates
  sns.set(style="white", color_codes=True)
  from matplotlib_venn import venn2
  import keras
  import pydot
  import libarchive





In [8]:
try:
  torch.__version__
  x = torch.empty(5, 3)
  print(x)
except:
  # Not sure why this has to be tried a 2nd time
  !pip3 install http://download.pytorch.org/whl/cpu/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
  !pip3 install torchvision
  try:
    torch.__version__
    x = torch.empty(5, 3)
    print(x)
  except:
    !pip3 install http://download.pytorch.org/whl/cpu/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
    !pip3 install torchvision
    
    
    
  


tensor([[ 3.8950e-35,  0.0000e+00,  4.4842e-44],
        [ 0.0000e+00,         nan,  0.0000e+00],
        [ 2.1296e+23,  1.3178e-08,  8.4483e+20],
        [ 6.5638e-07,  5.3644e-08,  1.0525e+21],
        [ 8.5456e-07,  5.2477e-08,  0.0000e+00]])


Consider the following:



```sql
select a.d,a.hr,strftime('%s',b.d)-strftime('%s',a.d),b.hr-a.hr,
(b.hr-a.hr)/(1.0*(strftime('%s',b.d)-strftime('%s',a.d)))
from hr a, hr b where a._rowid_ = b._rowid_ - 1;

```



---



In [9]:


from google.colab import auth
auth.authenticate_user()


project_id = 'mchirico'
!gcloud config set project {project_id}


!gsutil cp gs://heart-pigbot.appspot.com/hr.sqlite /tmp/hr.sqlite
 



Updated property [core/project].
Copying gs://heart-pigbot.appspot.com/hr.sqlite...
/ [1 files][ 10.0 MiB/ 10.0 MiB]                                                
Operation completed over 1 objects/10.0 MiB.                                     


In [69]:
import pandas as pd
import sqlite3
 
conn = sqlite3.connect('/tmp/hr.sqlite')
query = "SELECT _ROWID_,d,hr from hr;"
 
d = pd.read_sql_query(query,conn)
 
d.describe()

,rowid,hr
count,162452.000000,162452.000000
mean,81226.500000,85.233681
std,46895.997303,36.787001
min,1.000000,30.000000
25%,40613.750000,58.000000
50%,81226.500000,66.000000
75%,121839.250000,118.000000
max,162452.000000,215.000000


In [22]:

k=d[(d.d > '2018-07-12 00:00:00') & (d.hr > 130)]
k.set_index('d').diff()

,hr
d,
2018-07-12 09:20:30,NaN
2018-07-12 09:20:32,0.0
2018-07-12 09:23:11,-8.0
2018-07-12 09:23:16,0.0
2018-07-12 09:23:21,0.0
2018-07-12 09:23:36,0.0
2018-07-12 09:23:41,0.0
2018-07-12 09:23:51,0.0
2018-07-12 09:23:53,2.0


In [70]:
d.head(3)

,rowid,d,hr
0,1,2016-10-19 06:39:53,58
1,2,2016-10-19 06:56:57,52
2,3,2016-10-19 07:03:30,51


In [68]:
k=d[(d.d > '2018-07-12 00:00:00') & (d.hr > 130)]

k['dt']=pd.to_datetime(k['d'])

k['deltaTime'] = (k['dt']-k['dt'].shift()).fillna(0)
k['deltaHR'] = (k['hr']-k['hr'].shift()).fillna(0)
k['deltaSeconds']=k.deltaTime.dt.total_seconds()
#k.deltaTime.total_seconds()

k['rate']=k.deltaHR/k.deltaSeconds
k


,d,hr,dt,deltaTime,deltaHR,deltaSeconds,rate
161528,2018-07-12 09:20:30,139,2018-07-12 09:20:30,00:00:00,0.0,0.0,NaN
161529,2018-07-12 09:20:32,139,2018-07-12 09:20:32,00:00:02,0.0,2.0,0.000000
161554,2018-07-12 09:23:11,131,2018-07-12 09:23:11,00:02:39,-8.0,159.0,-0.050314
161555,2018-07-12 09:23:16,131,2018-07-12 09:23:16,00:00:05,0.0,5.0,0.000000
161556,2018-07-12 09:23:21,131,2018-07-12 09:23:21,00:00:05,0.0,5.0,0.000000
161558,2018-07-12 09:23:36,131,2018-07-12 09:23:36,00:00:15,0.0,15.0,0.000000
161559,2018-07-12 09:23:41,131,2018-07-12 09:23:41,00:00:05,0.0,5.0,0.000000
161561,2018-07-12 09:23:51,131,2018-07-12 09:23:51,00:00:10,0.0,10.0,0.000000
161562,2018-07-12 09:23:53,133,2018-07-12 09:23:53,00:00:02,2.0,2.0,1.000000
161563,2018-07-12 09:23:58,134,2018-07-12 09:23:58,00:00:05,1.0,5.0,0.200000
